### Start with importing class

In [187]:
from desummation import Desummation
import numpy as np
dsm = Desummation()

### Create new matrix
$$
    A = \begin{pmatrix}
    3 & 7 & 2 & 0 \\
    -4 & 2 & 0 & -3 \\
    5 & 0 & 2 & -1 \\
    5 & -5 & -2 & -4 \\
    \end{pmatrix} 
$$

In [188]:
A = [[3, 7, 2, 0], [-4, 2, 0, -3], [5, 0, 2, -1], [5, -5, -2, -4]] 

### Now fit some random matrices to this matrix

In [189]:
dsm.fit(A, 8)

### Get all the information you might need

In [191]:
dsm.weights()

[-0.7921641252757965,
 0.02856497634380073,
 0.8271760431198532,
 0.4419829614685806,
 0.32373200244346556,
 -3.3607544209767717,
 -2.4707198383725855,
 1.1102889503140112]

### Now let's see which matrix we got

In [192]:
dsm.predict(A)

array([[-2.61432699,  5.40926024,  1.98908676,  3.15317878],
       [-1.85441934,  4.47421948,  2.07227529, -0.47229592],
       [-1.07867   ,  6.90137977,  0.60671286, -1.49416506],
       [ 1.2140232 , -0.12356885, -2.25194497, -1.20161187]])

In [193]:
np.array(A)

array([[ 3,  7,  2,  0],
       [-4,  2,  0, -3],
       [ 5,  0,  2, -1],
       [ 5, -5, -2, -4]])

In [194]:
dsm.error()

21.635929305053782

### This is not very good approximation.
#### Certain ways to improve this:
- ##### make more trials for weights searching
- ##### make more random matrices
- ##### change metric between matrices
- ##### change distribution of elements in matrices

In [195]:
dsm.fit(A, 16, n_trials=2000, distribution='exponential', scale=1)

In [196]:
dsm.predict(A)

array([[ 2.49423493e+00,  7.03729570e+00,  1.52029639e+00,
         2.89002971e-03],
       [-4.82552532e+00,  2.56053925e+00, -2.54434113e-01,
        -4.02818656e+00],
       [ 5.33013779e+00,  7.46179565e-01,  2.69805038e+00,
        -8.14147507e-01],
       [ 3.61820494e+00, -4.10825540e+00, -2.27045590e+00,
        -2.56621879e+00]])

In [197]:
np.array(A)

array([[ 3,  7,  2,  0],
       [-4,  2,  0, -3],
       [ 5,  0,  2, -1],
       [ 5, -5, -2, -4]])

In [198]:
dsm.error()

2.9934800562620194

In [200]:
dsm.weights()

[-0.7416757946297725,
 -2.7044970637199057,
 -6.476469954411555,
 3.8006027499983297,
 2.9322448688949017,
 1.8536921265782187,
 4.289883674097096,
 0.32352551822984577,
 0.2155499512293888,
 -2.880591723501701,
 -1.2933023480717178,
 -6.066299519204691,
 -0.888029899842083,
 -0.8156192990179143,
 4.008491664386714,
 -0.02515789902350285]

# What are the inferences?

### You can experiment with it yourself, but judging by my intuition and my experiments:
- #### Almost always we will find weights with (frobenius norm) loss close to 0 `if` we use $n^2$ weights (tested only for square matrix for now). I think that it may be some sort of `solution to a linear system`. You may understand me beforehand if you remember the basis for matrices(the one consisting of $E_{ij}$ matrices)
- #### I think a lot about applying this to ML and DL and came up to some insight: [you can see this page](https://weightagnostic.github.io/) or trust my words. The experiment showcased on this page focused on utilizing random weights in ANNs `without` any explicit `weight training`, but `rather allowing the weights between neuron connections to be learned`. This approach offers a more cost-effective training method while still achieving impressive capabilities. This just reminds me of my problem.

## Now let's get some information about loss function for some fixed basis
### Is it convex? For someone who already knows the answer, you can go just down below for an explanation.

### First, start with simple one:
$$
A = \begin{pmatrix}
2 & -1 \\
-5 & 4 \\
\end{pmatrix}
$$


In [277]:
import plotly.graph_objects as go
dsm_convexity2 = Desummation()
A1 = [[2, -1], [-5, 4]]
dsm_convexity2.fit(A1, 2)

### Now define a function for plotting
- #### We will need a meshgrid of x and y.
    - ##### I will be using only 2 random matrices for good visualization (x and y coordinates).
    - ##### Weights are programmed to be found from $2 \cdot min$ to $2 \cdot max$ value of matrix A, but maybe this is wrong. I don't know it yet.
- #### Also a function that will return an error, for that I will calculate the frobenius norm loss.

In [274]:
n = 200
w1 = np.linspace(-20, 20, n)
w2 = np.linspace(-20, 20, n)

In [203]:
def frobenius_loss(x, y, dsm_object, target, distance='fro'):
    B = np.tensordot(np.stack([x, y]), dsm_object.matrices(), axes=1)
    return np.linalg.norm(target - B, ord=distance)

In [204]:
def plot(w1, w2, Z):
    fig = go.Figure(data=[go.Surface(x=w1, y=w2, z=Z)], layout=go.Layout(width=600, height=400))

    # Set layout options
    fig.update_layout(
        title='3D Plot of Loss Function',
        scene=dict(
            xaxis_title='w1',
            yaxis_title='w2',
            zaxis_title='Loss',
            aspectratio=dict(x=1, y=1, z=1),
            camera=dict(
                eye=dict(x=1, y=1, z=1)
            )
        ),
        autosize=True
    )
    argmin = np.argmin(Z)
    row_index = argmin // n
    col_index = argmin % n

    print(f'Minimum value obtained at {np.min(Z)} with weights: w1:{w1[col_index]} and w2:{w2[row_index]}')
    fig.show()


In [ ]:
Z = np.array([[frobenius_loss(x=x, y=y, dsm_object=dsm_convexity2, target=A1) for x in w1] for y in w2])
#plot(w1, w2, Z)

![plot1](plot1.png)

### I think it's a strike!!!
#### It's `clearly convex` function! And only with 2 weights. Case was very simple, but you can modify values in A, call dsm_convexety.fit() again to update random matrices, see if anything changes.

### Now let's try more difficult one:
#### This time convexity is quite questionable
$$
A = \begin{pmatrix}
2 & -5 & 4 \\
-3 & -3 & 2 \\
1 & 2 & 6 \\
\end{pmatrix}
$$


In [279]:
dsm_convexity3 = Desummation()
A2 = [[2, -5, 4], [-3, -3, 2], [1, 2, 6]]
dsm_convexity3.fit(A2, 2, distribution = 'normal')

In [280]:
n = 100
w1 = np.linspace(-12, 12, n)
w2 = np.linspace(-12, 12, n)

In [ ]:
Z = np.array([[frobenius_loss(x=x, y=y, dsm_object=dsm_convexity3, target=A2) for x in w1] for y in w2])
#plot(w1, w2, Z)

![plot2](plot2.png)

### This is convex too! You can try various shapes of A, various distributions, but there wouldn't be a case of not convex plot.
#### This is all because of convexity of frobenius norm!
So we can just go with how many weights we want and will always find a best approximation for our matrix.
## Now it's time for further experiments... Stay tuned!